# Trapped-Ion Quantum Computing Tools
Still very untested and mostly customized for my own purposes. I will be modifying it to be more generally usable.

In [1]:
using IonSim
using QuantumOptics: timeevolution, stochastic, projector

In [2]:
import PyPlot
const plt = PyPlot;
# set some plot configs
plt.matplotlib.rc("xtick", top=false)
plt.matplotlib.rc("ytick", right=false, left=false)
plt.matplotlib.rc("axes", labelsize=20, titlesize=20, grid=true)
plt.matplotlib.rc("axes", linewidth=2)
plt.matplotlib.rc("grid", alpha=0.25, linestyle="--")
plt.matplotlib.rc("font", family="Palatino", weight="medium")
plt.matplotlib.rc("figure", figsize=(8,4))
plt.matplotlib.rc("xtick.major", width=2)
plt.matplotlib.rc("ytick.major", width=2)

PySide6/__init__.py: Unable to import Shiboken from /home/vikk/Documents/Research/sqrlab/crosstalk, /home/vikk/Documents/Research/AXEAP/pyAXEAP, /usr/lib/python310.zip, /usr/lib/python3.10, /usr/lib/python3.10/lib-dynload, /home/vikk/.local/lib/python3.10/site-packages, /usr/lib/python3.10/site-packages

** (process:457785): WARNING **: 11:37:41.284: Failed to load shared library 'libgtk-4.so.1' referenced by the typelib: /usr/bin/../lib/julia/libstdc++.so.6: version `GLIBCXX_3.4.30' not found (required by /usr/bin/../lib/libicuuc.so.72)


## Constants

In [3]:
c = 299792458 # units of m/s
q_e = 1.60218e-19 # units of C
amu = 1.66054e-27 # units of kg
e0 = 8.854188e-12 # units of F/m
m_calcium = 40.078*amu # Default ion

6.655112212e-26

In [4]:
C = Ca40(["S-1/2", "D-1/2"])

⁴⁰Ca


## Ion Positions

In [5]:
using NLsolve

function IonPositionsV!(V, u)
    N = length(u)
    for m in 1:1:N
        V[m] = u[m] - sum([1/(u[m]-u[n])^2 for n in 1:1:m-1]) + sum([1/(u[m]-u[n])^2 for n in m+1:1:N])
    end
end

function calcScaledIonPositions(N::Integer)
    extreme = 0.481*N^0.765
    initial = collect(-extreme:2extreme/(N-1):extreme)
    res = nlsolve(IonPositionsV!,initial)
    return res.zero
end

lengthScale(ν,M,Z=1) = ((Z^2*q_e^2)/(4*pi*e0*M*ν^2))^(1/3)

lengthScale (generic function with 2 methods)

## Mode Calculation

In [6]:
using LinearAlgebra

In [7]:
function calcVibrationalModes(N)
    u = calcScaledIonPositions(N)
    A = [[m==n ? 1+2*sum([p!=m ? 1/abs(u[m]-u[p])^3 : 0 for p=1:N]) : -2/abs(u[m]-u[n])^3 for m=1:N] for n=1:N]
    A = permutedims(hcat(A...))
    eigendata = eigen(A)
    modes = [(eigendata.values[i],eigendata.vectors[:,i]) for i=1:N]
    return modes
end

calcVibrationalModes (generic function with 1 method)

## Crosstalk Calculation

In [8]:
# Laser pointed at particular ion, indicated by index
function calcEFieldCrosstalk(N::Integer, lasercenter::Integer, w::Real, ν::Real, M=m_calcium::Real) #
    p = calcScaledIonPositions(N).*lengthScale(ν, M) # positions
    c = p[lasercenter]
    scaledE = [ℯ^(-((p[n]-c)/w)^2) for n in 1:1:N]
    scaledE /= scaledE[1]
end

# Laser pointed at point, indicated by real number "index" (1.5 means halfway between ions 1 and 2)
function calcEFieldCrosstalk(N::Integer, lasercenter::Real, w::Real, ν::Real, M=m_calcium::Real) #
    p = calcScaledIonPositions(N).*lengthScale(ν, M) # positions
    left = Int(floor(lasercenter))
    if left == length(p)
        c = p[t]
    else
        c = (p[left+1]-p[left])*(lasercenter%1)+p[left]
    end
    scaledE = [ℯ^(-((p[n]-c)/w)^2) for n in 1:1:N]
    scaledE /= scaledE[1]
end

calcEFieldCrosstalk (generic function with 4 methods)

## Solution Analysis

In [9]:
function extractIonProbabilities(sol, T::Trap, ionindex, sublevel)
    IC = T.configuration
    modes = get_vibrational_modes(IC)
    N = length(T.configuration.ions)
    ion = IC.ions[ionindex]
    ions_to_trace_out = deleteat!(collect(1:N),ionindex)
    ρ_red = [ptrace(ψ ⊗ dagger(ψ), ions_to_trace_out) for ψ=sol]
    observable = projector(ion[sublevel]) ⊗ tensor([one(m) for m=modes]...)
    probs = expect(observable,ρ_red)
    return probs
end

extractIonProbabilities (generic function with 1 method)

In [10]:
function createBellState(N, ionindices)
    bellstate = normalize(tensor([C["S-1/2"] for i=1:N])
        + 1im*tensor([i in ionindices ? C["D-1/2"] : C["S-1/2"] for i=1:N]))
end

createBellState (generic function with 1 method)

In [11]:
function extractBellStateProbabilities(sol, T::Trap, ionindices)
    IC = T.configuration
    modes = get_vibrational_modes(IC)
    N = length(T.configuration.ions)
    bellstate = createBellState(N, ionindices)
    observable = projector(bellstate) ⊗ tensor([one(m) for m=modes]...)
    probs = abs.(expect(observable, sol))
    return probs
end

extractBellStateProbabilities (generic function with 1 method)

In [12]:
function extractModeProbabilities(sol, T::Trap, ionindex, modeindex, modelevel)
    IC = T.configuration
    modes = get_vibrational_modes(IC)
    N = length(T.configuration.ions)
    ion = IC.ions[ionindex]
    ions_to_trace_out = deleteat!(collect(1:N),ionindex)
    ρ_red = [ptrace(ψ ⊗ dagger(ψ), ions_to_trace_out) for ψ=sol]
    observable = one(ion) ⊗ tensor([one(m) for m=modes[1:modeindex-1]]) ⊗
        projector(modes[modelevel]) ⊗ tensor([one(m) for m=modes[modeindex+1:end]])
    probs = expect(observable,ρ_red)
    return probs
end

extractModeProbabilities (generic function with 1 method)

## Oscillation Period Estimation

In [13]:
modelf(x,T,ϕ) = (sin((2π*x/T)+ϕ)+1)/2
error(data,T,ϕ)=sum((modelf(x,T,ϕ)-data[x])^2 for x=1:length(data))
function estimateOscillationPeriod(times, signal, Trange, accuracy_level, ϕ)
    r = Trange
    errors = []
    lowesti = -1
    strafecount = 1000
    for level=1:accuracy_level
        println(r)
        if r.start==r.stop
            break
        end
        step = ((r.stop-r.start))/strafecount
        errors = [[real(T),real(error(signal,T,ϕ))] for T=r.start:step:r.stop]
        lowesti = findmin(real.(getindex.(errors,2)))[2]
        #println(findmin(real.(getindex.(errors,2))))
        #println(lowesti)
        r = r[Int(round(step*(max(1, lowesti-1)-1)))+1]:r[Int(round(step*(min(length(errors),lowesti+1)-1)))+1]
        #print(real(errors[lowesti][2]))
    end
    return errors[lowesti][1]*tout[end]/length(tout)
end

estimateOscillationPeriod (generic function with 1 method)

## Running Sims

In [14]:
function setMSLasers(T, target_modenum, pointing)
    L1, L2 = T.lasers
    L1.pointing = pointing
    L2.pointing = pointing
    
    # Set the laser parameters
    ϵ = 40e3
    d = 80  # corrects for AC stark shift from single-photon coupling to sidebands
    target_mode = T.configuration.vibrational_modes.z[1]
    Δf = transition_frequency(T, 1, ("S-1/2", "D-1/2"))
    L1.Δ = Δf + target_mode.ν + ϵ - d
    L1.k = ẑ
    L1.ϵ = x̂

    L2.Δ = Δf - target_mode.ν - ϵ + d
    L2.k = ẑ
    L2.ϵ = x̂
    L1.E = 43694.035471187046
    L2.E = 43694.035471187046
end    

function setEfield(T, target_modenum, target_ions)
    ionlist = T.configuration.ions
    target_mode = T.configuration.vibrational_modes.z[1]
    L1 = T.lasers[1]
    ionindex = target_ions[1]
    η = abs(get_η(target_mode, L1, ionlist[ionindex]))
    ϵ = 40e3
    Ω1 = √(1e3 * ϵ) / η  # This will give a 1kHz MS strength, since coupling goes like (ηΩ)^2/ϵ
    Efield_from_rabi_frequency!(Ω1, T, 1, ionindex, ("S-1/2", "D-1/2"))
    Efield_from_rabi_frequency!(Ω1, T, 2, ionindex, ("S-1/2", "D-1/2"));
end

function createMSTrap(N, target_modenum, target_ions)
    # if target_modenum == 1
    #     modenums = [1,2]
    # elseif target_modenum == N
    #     modenums = [N-1, N]
    # else
    #     modenums = [target_modenum-1, target_modenum, target_modenum+1]
    # end
    modenums = [target_modenum]
    ionlist = [copy(C) for i in 1:1:N];
    chain = LinearChain(
        ions=ionlist, com_frequencies=(x=3e6,y=3e6,z=1e6), 
        vibrational_modes=(x=[],y=[],z=modenums)
    )
    pointing = [i in target_ions ? (i,1.0) : (i,0.0) for i=1:N]
    T = Trap(configuration=chain, B=4e-4, Bhat=(x̂ + ẑ)/√2, lasers=[Laser(), Laser()])
    setMSLasers(T, target_modenum, pointing)
#     setEfield(T, target_modenum, target_ions)
    return T
end

function createInitialState(T)
    N = length(T.configuration.ions)
    ψ0 = ionstate(T, repeat(["S-1/2"],N)...) ⊗ tensor([mode[0] for mode=T.configuration.vibrational_modes.z]);  # initial state
end

function runSim(T, ψ0, tspan)
    h = hamiltonian(T, rwa_cutoff=5e5)
    tout, sol = timeevolution.schroedinger_dynamic(tspan, ψ0, h)
    return tout, sol
end

runSim (generic function with 1 method)